In [75]:
#%matplotlib inline
import pandas as pd
import numpy as np
import nltk
import string
import re
#import matplotlib.pyplot as plt
#import seaborn as sns
#from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, roc_curve, auc,precision_score,recall_score,f1_score
from sklearn import metrics
#from nltk.stem.porter import PorterStemmer
#import string
#from nltk.corpus import stopwords
#from nltk.stem import PorterStemmer
#from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [76]:
import pandas as pd
data=pd.read_csv('C:/Users/Khushboo/Documents/datasets/SMSSpamCollection',delimiter="\t",header=None,names=["class","Text"])

In [77]:
print('Number of samples in the dataset is {} and features {}'.format(data.shape[0], data.shape[1]))

Number of samples in the dataset is 5572 and features 2


In [78]:
print(data.head())

  class                                               Text
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [79]:
data.duplicated().sum()

403

In [80]:
#Deduplication of entries
final_data = data.drop_duplicates(subset={"class","Text"}, keep='first', inplace=False)
print('Number of samples in the dataset after removing duplicates is {} and features {}'.format(final_data.shape[0], final_data.shape[1]))

Number of samples in the dataset after removing duplicates is 5169 and features 2


In [81]:
print('Number of samples in the dataset after removing duplicates is {} and features {}'.format(final_data.shape[0], final_data.shape[1]))

Number of samples in the dataset after removing duplicates is 5169 and features 2


In [82]:
print(final_data['class'].value_counts())

ham     4516
spam     653
Name: class, dtype: int64


In [83]:
final_data.duplicated().sum()

0

In [84]:
final_data.head()

,class,Text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [85]:
tmp  = []
for i in range(data.shape[0]):
    if data['class'][i] == 'ham':
        tmp.append(1)
    else:
        tmp.append(0)

In [86]:
final_data = data.drop('class', axis=1)

In [87]:
final_data['class'] = tmp

In [88]:
final_data.head()

,Text,class
0,"Go until jurong point, crazy.. Available only ...",1
1,Ok lar... Joking wif u oni...,1
2,Free entry in 2 a wkly comp to win FA Cup fina...,0
3,U dun say so early hor... U c already then say...,1
4,"Nah I don't think he goes to usf, he lives aro...",1


In [89]:
final_data.shape

(5572, 2)

In [90]:
sno = nltk.stem.SnowballStemmer('english')
def cleanpunc(sentence): 
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned

In [91]:
i=0
str1=' '
final_string=[]
spam=[]
ham=[]
s=''

for w in final_data['Text'].values:
    filtered_sentence=[]
    for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                s=(sno.stem(cleaned_words.lower())).encode('utf8')
                filtered_sentence.append(s)
                if (final_data['class'].values)[i] == 1: 
                    spam.append(s)
                if(final_data['class'].values)[i] == 0:
                   ham.append(s)
            else:
                continue 
    str1 = b" ".join(filtered_sentence) 
    final_string.append(str1)
    i+=1

In [92]:
final_data['CleanedText']=final_string 

In [93]:
final_data = final_data.drop('Text',axis=1)
final_data.head()

,class,CleanedText
0,1,b'until jurong point crazi avail onli bugi gre...
1,1,b'lar joke wif oni'
2,0,b'free entri wkli comp win cup final tkts may ...
3,1,b'dun say earli hor alreadi then say'
4,1,b'nah dont think goe usf live around here though'


In [94]:
final_data.shape

(5572, 2)

In [95]:
final_data.drop_duplicates(subset={"CleanedText"}, keep='first', inplace=True)

In [96]:
final_data.shape

(5052, 2)

In [97]:
print(final_data['class'].value_counts())

1    4480
0     572
Name: class, dtype: int64


In [98]:
ham = final_data[final_data['class'] == 1]
spam = final_data[final_data['class'] == 0]

In [99]:
ham = ham[:600]

In [100]:
data = pd.concat([ham, spam])
data.shape

(1172, 2)

In [101]:
data.head(10)

,class,CleanedText
0,1,b'until jurong point crazi avail onli bugi gre...
1,1,b'lar joke wif oni'
3,1,b'dun say earli hor alreadi then say'
4,1,b'nah dont think goe usf live around here though'
6,1,b'even brother not like speak with they treat ...
7,1,b'per your request mell mell oru minnaminungin...
10,1,b'gonna home soon and dont want talk about thi...
13,1,b'ive been search for the right word thank you...
14,1,b'have date sunday with will'
16,1,b'watch'


In [102]:
data.tail(10)

,class,CleanedText
5462,0,b'call claim your reward hour talk time use fr...
5466,0,b'http tms widel com index'
5468,0,b'urgent last weekend draw show that you have ...
5482,0,b'urgent are tri contact you last weekend draw...
5487,0,b'per min call germani from your line just per...
5492,0,b'marvel mobil play the offici ultim game mobi...
5537,0,b'want explicit sex sec ring now cost min gsex...
5540,0,b'ask chatlin inclu free min india cust serv s...
5566,0,b'remind from get pound free call credit and d...
5567,0,b'this the time have tri contact have won the ...


In [103]:
from sklearn.utils import shuffle
data = shuffle(data)
data.head()

,class,CleanedText
575,1,b'nope wait sch daddi'
4903,0,b'polyphon rington text super get your free po...
468,1,b'when are you go ride your bike'
115,1,b'openin sentenc veri formal anyway fine too j...
457,1,b'look ami ure beauti intellig woman and like ...


In [104]:
data1 = data['CleanedText']
label = data['class']

In [105]:
X_train,X_test,y_train,y_test = train_test_split(data1,label,test_size=0.3,random_state=4)

In [106]:
from sklearn.feature_extraction.text import CountVectorizer

In [107]:
vect = CountVectorizer()

In [108]:
data_dtm = vect.fit(data1)
X_train = vect.transform(X_train)
X_test = vect.transform(X_test)

In [109]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

In [110]:
k_range= range(2,31)
k_scores = []

In [111]:
for k in k_range:
    knn= KNeighborsClassifier(n_neighbors=k)
    scores=cross_val_score(knn,X_train, y_train,cv=10, scoring="accuracy")
    k_scores.append(scores.mean())
print(k_scores)

[0.8000065301857837, 0.7084576789542997, 0.718229557797586, 0.6524718567132124, 0.6647874243133327, 0.6281532634603455, 0.6342812986362796, 0.6038049215833524, 0.6111671618724219, 0.5927687262147052, 0.5940032941159398, 0.5756489372122637, 0.5818067210123239, 0.5635118613569, 0.5647166806340085, 0.5451876158654492, 0.552505414612379, 0.5366796907594242, 0.5415581023280112, 0.5354598157762032, 0.5366646350533116, 0.5293762221424088, 0.5317858606966257, 0.5232632426725873, 0.5244680619496956, 0.5196043432991223, 0.5208389112003569, 0.5146660716941841, 0.5146660716941841]


In [112]:
optimal_k = k_range[k_scores.index(max(k_scores))]
optimal_k

2

In [113]:
knn = KNeighborsClassifier(n_neighbors=optimal_k)
knn.fit(X_train, y_train) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=2, p=2,
           weights='uniform')

In [114]:
y_pred = knn.predict(X_test)

In [115]:
accuracy_score(y_test,y_pred)

0.8125

In [119]:
confusion_matrix(y_test,y_pred)

array([[104,  62],
       [  4, 182]], dtype=int64)

In [120]:
precision_score(y_test, y_pred)  

0.7459016393442623

In [121]:
recall_score(y_test, y_pred)

0.978494623655914

In [122]:
f1_score(y_test, y_pred)

0.8465116279069768